In [1]:

from pathlib import Path

import iris
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
tmpdir = Path('/work/scratch-nopw2/mmuetz/wcrp_hackathon/tmp_iris_conversion')

In [3]:
ds2d = xr.open_dataset(tmpdir / 'glm.n2560_RAL3p3.2D_20200120T00.nc')
ds3d_p = xr.open_dataset(tmpdir / 'glm.n2560_RAL3p3.3Dpressure_20200120T00.nc')
ds3d_ml = xr.open_dataset(tmpdir / 'glm.n2560_RAL3p3.3Dmodellevel_20200120T00.nc')

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_4032/2365851045.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds2d = xr.open_dataset(tmpdir / 'glm.n2560_RAL3p3.2D_20200120T00.nc')
/tmp/ipykernel_4032/2365851045.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds2d = xr.open_dataset(tmpdir / 'glm.n2560_RAL3p3.2D_20200120T00.nc')
/tmp/ipykernel_4032/2365851045.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta t

In [4]:
ds2d

<xarray.Dataset> Size: 23GB
Dimensions:                                                       (time: 13,
                                                                   latitude: 3840,
                                                                   longitude: 5120,
                                                                   time_0: 13,
                                                                   latitude_0: 3841,
                                                                   longitude_0: 5120,
                                                                   time_1: 12,
                                                                   bnds: 2,
                                                                   time_2: 12,
                                                                   time_3: 12)
Coordinates: (12/17)
  * time                                                          (time) datetime64[ns] 104B ...
  * latitude                                                      (latitude) float32 15kB ...
  * longitude                                                     (longitude) float32 20kB ...
    forecast_period                                               (time) timedelta64[ns] 104B ...
    forecast_reference_time                                       datetime64[ns] 8B ...
  * time_0                                                        (time_0) datetime64[ns] 104B ...
    ...                                                            ...
  * time_1                                                        (time_1) datetime64[ns] 96B ...
    forecast_period_1                                             (time_1) timedelta64[ns] 96B ...
  * time_2                                                        (time_2) datetime64[ns] 96B ...
    forecast_period_2                                             (time_2) timedelta64[ns] 96B ...
  * time_3                                                        (time_3) datetime64[ns] 96B ...
    forecast_period_3                                             (time_3) timedelta64[ns] 96B ...
Dimensions without coordinates: bnds
Data variables: (12/27)
    air_pressure_at_sea_level                                     (time, latitude, longitude) float32 1GB ...
    latitude_longitude                                            int32 4B ...
    air_temperature                                               (time_0, latitude, longitude) float32 1GB ...
    atmosphere_cloud_liquid_water_content                         (time, latitude, longitude) float32 1GB ...
    atmosphere_cloud_ice_content                                  (time, latitude, longitude) float32 1GB ...
    m01s30i461                                                    (time, latitude, longitude) float32 1GB ...
    ...                                                            ...
    surface_temperature                                           (time, latitude, longitude) float32 1GB ...
    toa_incoming_shortwave_flux                                   (time_2, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux                                   (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux_assuming_clear_sky                (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux                                    (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux_assuming_clear_sky                 (time_3, latitude, longitude) float32 944MB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [5]:
def interp_to_standard_grid(ds, varname):
    lat = ds.latitude
    lon = ds.longitude
    da = ds[varname]
    coords = {c: da[c] for c in da.dims[:-2]}
    
    #time_coord = ds[[c for c in da.coords if c.startswith('time')][0]]
    lat_coord_name = [c for c in da.coords if c.startswith('latitude')][0]
    lon_coord_name = [c for c in da.coords if c.startswith('longitude')][0]
    coords['latitude'] = lat
    coords['longitude'] = lon
    data = np.zeros([*da.shape[:-2]] + [len(lat), len(lon)])
    new_da = xr.DataArray(
        data,
        name=da.name,
        coords=coords,
    )
    new_da.values = da.interp(**{lat_coord_name:lat, lon_coord_name:lon}).values
    ds[varname] = new_da
    

In [6]:
interp_to_standard_grid(ds2d, 'x_wind')
interp_to_standard_grid(ds2d, 'y_wind')

/tmp/ipykernel_4032/2080186110.py:18: SerializationWarning: Can't decode floating point timedelta to 's' without precision loss, decoding to 'ns' instead. To silence this warning use time_unit='ns' in call to decoding function.
  new_da.values = da.interp(**{lat_coord_name:lat, lon_coord_name:lon}).values
/tmp/ipykernel_4032/2080186110.py:18: SerializationWarning: Can't decode floating point timedelta to 's' without precision loss, decoding to 'ns' instead. To silence this warning use time_unit='ns' in call to decoding function.
  new_da.values = da.interp(**{lat_coord_name:lat, lon_coord_name:lon}).values


In [10]:
drop_vars = [
    'latitude_0',
    'longitude_0',
    #'bnds',
    'forecast_period',
    'forecast_reference_time',
    'forecast_period_0',
    'height',
    'height_0',
    'forecast_period_1',
    'forecast_period_2',
    'forecast_period_3',
    'latitude_longitude',
    'time_1_bnds',
    'forecast_period_1_bnds',
]
    
    

ds2d = ds2d.drop_vars(drop_vars)
ds2d

<xarray.Dataset> Size: 23GB
Dimensions:                                                       (time: 13,
                                                                   latitude: 3840,
                                                                   longitude: 5120,
                                                                   time_0: 13,
                                                                   time_1: 12,
                                                                   time_2: 12,
                                                                   time_3: 12)
Coordinates:
  * time                                                          (time) datetime64[ns] 104B ...
  * latitude                                                      (latitude) float32 15kB ...
  * longitude                                                     (longitude) float32 20kB ...
  * time_0                                                        (time_0) datetime64[ns] 104B ...
  * time_1                                                        (time_1) datetime64[ns] 96B ...
  * time_2                                                        (time_2) datetime64[ns] 96B ...
  * time_3                                                        (time_3) datetime64[ns] 96B ...
Data variables: (12/24)
    air_pressure_at_sea_level                                     (time, latitude, longitude) float32 1GB ...
    air_temperature                                               (time_0, latitude, longitude) float32 1GB ...
    atmosphere_cloud_liquid_water_content                         (time, latitude, longitude) float32 1GB ...
    atmosphere_cloud_ice_content                                  (time, latitude, longitude) float32 1GB ...
    m01s30i461                                                    (time, latitude, longitude) float32 1GB ...
    cloud_area_fraction_assuming_maximum_random_overlap           (time_0, latitude, longitude) float32 1GB ...
    ...                                                            ...
    surface_temperature                                           (time, latitude, longitude) float32 1GB ...
    toa_incoming_shortwave_flux                                   (time_2, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux                                   (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux_assuming_clear_sky                (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux                                    (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux_assuming_clear_sky                 (time_3, latitude, longitude) float32 944MB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [9]:
ds2d

<xarray.Dataset> Size: 23GB
Dimensions:                                                       (time: 13,
                                                                   latitude: 3840,
                                                                   longitude: 5120,
                                                                   time_0: 13,
                                                                   latitude_0: 3841,
                                                                   longitude_0: 5120,
                                                                   time_1: 12,
                                                                   bnds: 2,
                                                                   time_2: 12,
                                                                   time_3: 12)
Coordinates: (12/17)
  * time                                                          (time) datetime64[ns] 104B ...
  * latitude                                                      (latitude) float32 15kB ...
  * longitude                                                     (longitude) float32 20kB ...
    forecast_period                                               (time) timedelta64[ns] 104B ...
    forecast_reference_time                                       datetime64[ns] 8B ...
  * time_0                                                        (time_0) datetime64[ns] 104B ...
    ...                                                            ...
  * time_1                                                        (time_1) datetime64[ns] 96B ...
    forecast_period_1                                             (time_1) timedelta64[ns] 96B ...
  * time_2                                                        (time_2) datetime64[ns] 96B ...
    forecast_period_2                                             (time_2) timedelta64[ns] 96B ...
  * time_3                                                        (time_3) datetime64[ns] 96B ...
    forecast_period_3                                             (time_3) timedelta64[ns] 96B ...
Dimensions without coordinates: bnds
Data variables: (12/27)
    air_pressure_at_sea_level                                     (time, latitude, longitude) float32 1GB ...
    latitude_longitude                                            int32 4B ...
    air_temperature                                               (time_0, latitude, longitude) float32 1GB ...
    atmosphere_cloud_liquid_water_content                         (time, latitude, longitude) float32 1GB ...
    atmosphere_cloud_ice_content                                  (time, latitude, longitude) float32 1GB ...
    m01s30i461                                                    (time, latitude, longitude) float32 1GB ...
    ...                                                            ...
    surface_temperature                                           (time, latitude, longitude) float32 1GB ...
    toa_incoming_shortwave_flux                                   (time_2, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux                                   (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux_assuming_clear_sky                (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux                                    (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux_assuming_clear_sky                 (time_3, latitude, longitude) float32 944MB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [11]:
import sys
sys.path.insert(0, '/home/users/mmuetz/deploy/global_hackathon_tools/dataset_transforms')


In [12]:
from um_latlon_pp_to_healpix_nc import UMRegridder


In [13]:
regridder = UMRegridder('easygems_delaunay')

In [14]:
regridded_data, dim_shape, dim_ranges = regridder.regrid(ds2d.x_wind)

/home/users/mmuetz/deploy/global_hackathon_tools/dataset_transforms/um_latlon_pp_to_healpix_nc.py:177: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f'  - {dict(dsout_tpl.dims)}')


  - {'time_0': 13}
  - (0,)
  - (1,)
  - (2,)
  - (3,)
  - (4,)
  - (5,)
  - (6,)
  - (7,)
  - (8,)
  - (9,)
  - (10,)
  - (11,)
  - (12,)


In [ ]:
ds2dsmall = ds2d[['air_pressure_at_sea_level', 'x_wind']].copy()

In [ ]:
ds2dsmall

In [ ]:
interp_to_standard_grid(ds2dsmall, 'x_wind')

In [ ]:
ds2dsmall

In [ ]:
from functools import partial

In [ ]:
f_x_wind = partial(interp_to_standard_grid, varname='x_wind')
ds2dsmall = ds[['x_wind']].copy()
f_x_wind(ds2dsmall)
ds2dsmall

In [56]:
ds3d_p

<xarray.Dataset> Size: 69GB
Dimensions:                  (time: 5, pressure: 25, latitude: 3841,
                              longitude: 5120, latitude_0: 3840,
                              longitude_0: 5120)
Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * latitude                 (latitude) float32 15kB -90.0 -89.95 ... 89.95 90.0
  * longitude                (longitude) float32 20kB 0.0 0.07031 ... 359.9
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...
  * latitude_0               (latitude_0) float32 15kB -89.98 -89.93 ... 89.98
  * longitude_0              (longitude_0) float32 20kB 0.03516 0.1055 ... 360.0
Data variables:
    x_wind                   (time, pressure, latitude, longitude) float32 10GB ...
    latitude_longitude       int32 4B ...
    geopotential_height      (time, pressure, latitude_0, longitude_0) float32 10GB ...
    y_wind                   (time, pressure, latitude, longitude) float32 10GB ...
    relative_humidity        (time, pressure, latitude, longitude) float32 10GB ...
    relative_humidity_0      (time, pressure, latitude_0, longitude_0) float32 10GB ...
    air_temperature          (time, pressure, latitude_0, longitude_0) float32 10GB ...
    upward_air_velocity      (time, pressure, latitude_0, longitude_0) float32 10GB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [6]:
ds3d_psmall = ds3d_p[['x_wind', 'geopotential_height']].copy()

In [58]:
#interp_to_standard_grid(ds3d_psmall, 'x_wind')

In [59]:
ds3d_psmall.x_wind.coords

Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * latitude                 (latitude) float32 15kB -90.0 -89.95 ... 89.95 90.0
  * longitude                (longitude) float32 20kB 0.0 0.07031 ... 359.9
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...

In [ ]:
ds = ds3d_psmall.copy()
varname = 'x_wind'

lat = ds.latitude
lon = ds.longitude
da = ds[varname]
coords = {c: da[c] for c in da.dims[:-2]}

#time_coord = ds[[c for c in da.coords if c.startswith('time')][0]]
lat_coord_name = [c for c in da.coords if c.startswith('latitude')][0]
lon_coord_name = [c for c in da.coords if c.startswith('longitude')][0]
coords['latitude'] = lat
coords['longitude'] = lon
data = np.zeros([*da.shape[:-2]] + [len(lat), len(lon)])
new_da = xr.DataArray(
    data,
    name=da.name,
    coords=coords,
)
new_da.values = da.interp(**{lat_coord_name:lat, lon_coord_name:lon}).values
ds[varname] = new_da

In [ ]:
import numpy as np
ds = ds3d_psmall.copy()
varname = 'x_wind'

lat = ds.latitude
lon = ds.longitude
da = ds[varname]
coords = {c: da[c] for c in da.dims[:-2]}

#time_coord = ds[[c for c in da.coords if c.startswith('time')][0]]
lat_coord_name = [c for c in da.coords if c.startswith('latitude')][0]
lon_coord_name = [c for c in da.coords if c.startswith('longitude')][0]
coords['latitude'] = lat
coords['longitude'] = lon
data = da.values
new_data = data[:-1] + np.roll(data[:-1], 1, axis=-1) + data[1:] + np.roll(data[1:], 1, axis=-1)
new_da = xr.DataArray(
    new_data,
    name=da.name,
    coords=coords,
)

ds[varname] = new_da

In [215]:
da.coords

Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * latitude                 (latitude) float32 15kB -90.0 -89.95 ... 89.95 90.0
  * longitude                (longitude) float32 20kB 0.0 0.07031 ... 359.9
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...

In [216]:
da.dims

('time', 'pressure', 'latitude', 'longitude')

In [16]:
# These are taken straight from the spreadsheet - no mapping is done to actual names (yet)...

vars_2d = set('''atmosphere_mass_content_of_cloud_condensed_water
atmosphere_mass_content_of_cloud_ice
surface_downward_latent_heat_flux
surface_downward_sensible_heat_flux
toa_outgoing_longwave_flux
toa_outgoing_longwave_flux_clear_sky
toa_incoming_longwave_flux
surface_upwelling_longwave_flux_in_air
surface_upwelling_longwave_flux_in_air_clear_sky
surface_downwelling_longwave_flux_in_air
surface_downwelling_longwave_flux_in_air_clear_sky
toa_outgoing_shortwave_flux
toa_outgoing_shortwave_flux_clear_sky
toa_incoming_shortwave_flux
surface_upwelling_shortwave_flux_in_air
surface_upwelling_shortwave_flux_in_air_clear_sky
surface_downwelling_shortwave_flux_in_air
surface_downwelling_shortwave_flux_in_air_clear_sky
precipitation_flux
solid_precipitation_flux
atmosphere_mass_content_of_water_vapor
surface_air_pressure
air_pressure_at_mean_sea_level
specific_humidity
air_temperature
eastward_wind
northward_wind
x_wind
y_wind
surface_temperature
surface_downward_eastward_stress
surface_downward_northward_stress
cloud_area_fraction
liquid_water_content_of_surface_snow
snow_area_fraction_viewable_from_above
soil_liquid_water_content
sea_ice_area_fraction'''.split('\n'))


In [5]:
vars_3d = set("""geopotential height
eastward_wind
northtward_wind
upward_air_velocity
temperature
relative_humidity
specific_humidity
mass_fraction_of_cloud_liquid_water_in_air
mass_fraction_of_cloud_ice_in_air
mass_fraction_of_rain_in_air
mass_fraction_of_snow_water_in_air
mass_fraction_of_graupel_in_air""".split('\n'))


In [9]:
dss = {s.name: xr.open_dataset(p) for s, p in zip(streams, ncoutputs)}
#dss = = [xr.open_dataset(p) for p in ncoutputs]

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_4950/2340897413.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dss = {s.name: xr.open_dataset(p) for s, p in zip(streams, ncoutputs)}
/tmp/ipykernel_4950/2340897413.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dss = {s.name: xr.open_dataset(p) for s, p in zip(streams, ncoutputs)}
/tmp/ipykernel_4950/2340897413.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta

In [78]:
dss['apverd.pp']

<xarray.Dataset> Size: 20GB
Dimensions:                  (time: 5, pressure: 25, latitude: 3840,
                              longitude: 5120)
Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * latitude                 (latitude) float32 15kB -89.98 -89.93 ... 89.98
  * longitude                (longitude) float32 20kB 0.03516 0.1055 ... 360.0
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...
Data variables:
    air_temperature          (time, pressure, latitude, longitude) float32 10GB ...
    latitude_longitude       int32 4B ...
    relative_humidity        (time, pressure, latitude, longitude) float32 10GB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [53]:
all_varnames = [v for ds in dss.values() for v in ds.data_vars]

In [54]:
v2d = set(all_varnames) & vars_2d
v2d

{'air_temperature',
 'sea_ice_area_fraction',
 'specific_humidity',
 'surface_air_pressure',
 'surface_downwelling_longwave_flux_in_air',
 'surface_downwelling_shortwave_flux_in_air',
 'surface_temperature',
 'toa_incoming_shortwave_flux',
 'toa_outgoing_longwave_flux',
 'toa_outgoing_shortwave_flux'}

In [55]:
v3d = set(all_varnames) & vars_3d
v3d

{'mass_fraction_of_cloud_ice_in_air',
 'mass_fraction_of_cloud_liquid_water_in_air',
 'mass_fraction_of_graupel_in_air',
 'mass_fraction_of_rain_in_air',
 'relative_humidity',
 'specific_humidity',
 'upward_air_velocity'}

In [38]:
def get_var_details(ds, vname):
    if vname in ds:
        name = vname
        v = ds[vname]
        units = v.attrs['units']
        dtime = v.time.values[1] - v.time.values[0]
        return name, units, dtime.astype(int) / 1e9 / 3600

In [37]:
get_var_details(dss[0], 'air_temperature')

('air_temperature', 'K', np.float64(3480.0))

In [46]:
# Copied straight from excel. Note tab separated.
d = """air_pressure_at_mean_sea_level	psl	Pa
air_temperature	tas	K
atmosphere_mass_content_of_cloud_condensed_water	clwvi	kg m-2
atmosphere_mass_content_of_cloud_ice	clivi	kg m-2
atmosphere_mass_content_of_water_vapor	prw	kg m-2 s-1
cloud_area_fraction	clt	1
eastward_wind	uas	m s-1
liquid_water_content_of_surface_snow	swe	kg m-2
northward_wind	vas	m s-1
precipitation_flux	pr	kg m-2 s-1
sea_ice_area_fraction	siconc	1
snow_area_fraction_viewable_from_above	sncvfa	1
soil_liquid_water_content	mrso	kg m-2
solid_precipitation_flux	prs	kg m-2 s-1
specific_humidity	huss	kg kg-1
surface_air_pressure	ps	Pa
surface_downward_eastward_stress	tauu	N m-2
surface_downward_latent_heat_flux	hflsd	W m-2
surface_downward_northward_stress	tauv	N m-2
surface_downward_sensible_heat_flux	hfssd	W m-2
surface_downwelling_longwave_flux_in_air	rldt	W m-2
surface_downwelling_longwave_flux_in_air_clear_sky	rldscs	W m-2
surface_downwelling_shortwave_flux_in_air	rsds	W m-2
surface_downwelling_shortwave_flux_in_air_clear_sky	rsdscs	W m-2
surface_temperature	ts	K
surface_upwelling_longwave_flux_in_air	rlus	W m-2
surface_upwelling_longwave_flux_in_air_clear_sky	rluscs	W m-2
surface_upwelling_shortwave_flux_in_air	rsus	W m-2
surface_upwelling_shortwave_flux_in_air_clear_sky	rsuscs	W m-2
toa_incoming_longwave_flux	rldt	W m-2
toa_incoming_shortwave_flux	rsdt	W m-2
toa_outgoing_longwave_flux	rlut	W m-2
toa_outgoing_longwave_flux_clear_sky	rlutcs	W m-2
toa_outgoing_shortwave_flux	rsut	W m-2
toa_outgoing_shortwave_flux_clear_sky	rsutcs	W m-2"""
# This is tab separated!
lines = [l.split('\t') for l in d.split('\n')]
df2d  = pd.DataFrame(lines, columns=['cfname', 'short_name', 'units'])

In [48]:
import pandas as pd


In [49]:
d = """geopotential height	zg	m
eastward_wind	ua	m s-1
northtward_wind	va	m s-1
upward_air_velocity	wa	m s-1
temperature	ta	K
relative_humidity	hur	-
specific_humidity	hus	kg kg-1
mass_fraction_of_cloud_liquid_water_in_air	clw	kg kg-1
mass_fraction_of_cloud_ice_in_air	cli	kg kg-1
mass_fraction_of_rain_in_air	qr	kg kg-1
mass_fraction_of_snow_water_in_air	qs	kg kg-1
mass_fraction_of_graupel_in_air	qg	kg kg-1"""
lines = [l.split('\t') for l in d.split('\n')]
df3d  = pd.DataFrame(lines, columns=['cfname', 'short_name', 'units'])
df3d


,cfname,short_name,units
0,geopotential height,zg,m
1,eastward_wind,ua,m s-1
2,northtward_wind,va,m s-1
3,upward_air_velocity,wa,m s-1
4,temperature,ta,K
5,relative_humidity,hur,-
6,specific_humidity,hus,kg kg-1
7,mass_fraction_of_cloud_liquid_water_in_air,clw,kg kg-1
8,mass_fraction_of_cloud_ice_in_air,cli,kg kg-1
9,mass_fraction_of_rain_in_air,qr,kg kg-1
